In [ ]:
import sys
import os
import shutil
import pathlib
path_root = pathlib.Path(os.getcwd()).parents[1]
path_module = pathlib.Path(path_root) / 'chiral-client-python'
sys.path.append(str(path_module))
import chiral_client
print(f'chiral client version: {chiral_client.version}')
# register a user at cloud.chiral.one, get the token api from the page "Profile"
user_email = os.environ['CHIRAL_USER_EMAIL']
token_api = os.environ['CHIRAL_TOKEN_API']

In [ ]:
def create_client() -> chiral_client.Client:
    chiral_computing_host = "133.242.237.70"
    chiral_computing_port = "20000"
    return chiral_client.Client(user_email, token_api, chiral_computing_host, chiral_computing_port)

cc = create_client()

In [ ]:
simulation_id = 'lysozyme'
data_dir = f'{os.environ["CHIRAL_DATA_DIR"]}/gromacs/{simulation_id}'
local_dir = simulation_id
if os.path.exists(local_dir):
    shutil.rmtree(local_dir)
os.mkdir(local_dir)
job_mgr = chiral_client.GromacsJobManager(simulation_id, local_dir, cc)

# remove files under the existing simulation directory 
job_mgr.clear_files(cc)

# from pdb to topology
files_input = ["1AKI_clean.pdb"]
for filename in files_input:
    shutil.copy(f'{data_dir}/{filename}', f'{local_dir}/{filename}')
files_output = ["1AKI_processed.gro", "topol.top", "posre.itp"] 

job_mgr.upload_files(cc, ['1AKI_clean.pdb'])
job_id = job_mgr.submit_job(cc,
    'pdb2gmx', "-f 1AKI_clean.pdb -o 1AKI_processed.gro -water spce", "15",
    files_input, files_output
)
print(f'Job {job_id} created')
cc.wait_until_completion(job_id)
(output, error) = job_mgr.get_output(cc, job_id)
assert len(output) > 0
assert error == ''
job_mgr.download_files(cc, files_output)
for filename in files_output:
    assert os.path.exists(f'{local_dir}/{filename}')

shutil.rmtree(local_dir)
